<a href="https://colab.research.google.com/github/k-dinakaran/Intelligent-Hiring-Assistant-Chatbot-for-Recruitment-Agency/blob/main/hiring_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
!pip install gradio transformers torch accelerate bitsandbytes

In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
from datetime import datetime

# Initialize model and conversation state
model, tokenizer = None, None
conversation_state = {
    'user_data': {},
    'current_step': 0,
    'generated_questions': [],
    'question_answers': [],
    'awaiting_answers': False
}

information_steps = [
    ("What is your full name?", "name"),
    ("Please provide your email address:", "email"),
    ("What is your phone number?", "phone"),
    ("How many years of experience do you have? (0-1, 1-3, 3-5, 5+)", "experience"),
    ("What position are you applying for?", "position"),
    ("Where are you currently located?", "location"),
    ("Please list your technical skills (comma separated, e.g., Python, SQL):", "tech_stack")
]

def load_model():
    global model, tokenizer
    if model is None:
        model_name = "mistralai/Mistral-7B-Instruct-v0.1"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            load_in_4bit=True
        )
    return model, tokenizer

def generate_questions(tech_stack, experience):
    model, tokenizer = load_model()
    try:
        prompt = f"""Generate 5 technical questions for each skill in: {tech_stack}.
        For a candidate with {experience} years experience, create:
        1. Conceptual question
        2. Practical application
        3. Coding challenge
        4. Advanced problem
        5. Best practices

        Return ONLY the questions in this exact format:
        [Technology]:
        1. [Question]
        2. [Question]
        3. [Question]
        4. [Question]
        5. [Question]"""

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        output = model.generate(
            **inputs,
            max_new_tokens=1200,
            temperature=0.7,
            do_sample=True
        )
        return tokenizer.decode(output[0], skip_special_tokens=True)
    except Exception as e:
        return f"⚠️ Error generating questions: {str(e)}"

def parse_questions(question_text):
    """Convert the generated questions into a list of (tech, question) pairs"""
    questions = []
    current_tech = None
    for line in question_text.split('\n'):
        if line.endswith(':'):
            current_tech = line[:-1]
        elif line.strip() and line[0].isdigit():
            q_num = line.split('.')[0]
            question = line.split('. ')[1] if '. ' in line else line
            questions.append((current_tech, question))
    return questions

def save_interview_data(user_data, qa_pairs):
    """Save all interview data to a JSON file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"interview_{user_data['name'].replace(' ', '_')}_{timestamp}.json"

    data = {
        "candidate_info": user_data,
        "questions_answers": qa_pairs,
        "timestamp": timestamp
    }

    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

    return filename

def respond(message, chat_history):
    global conversation_state

    # Initialize conversation
    if not chat_history:
        conversation_state = {
            'user_data': {},
            'current_step': 0,
            'generated_questions': [],
            'question_answers': [],
            'awaiting_answers': False
        }
        return information_steps[0][0], False, False

    # Handle exit commands
    if message.lower() in ["exit", "quit", "bye", "goodbye"]:
        return "Thank you for your time!", False, False

    # If we're collecting answers to questions
    if conversation_state['awaiting_answers']:
        # Store the answer with its question
        current_q = conversation_state['generated_questions'][len(conversation_state['question_answers'])]
        conversation_state['question_answers'].append((current_q[1], message))

        # Check if all questions are answered
        if len(conversation_state['question_answers']) == len(conversation_state['generated_questions']):
            # Save to file
            filename = save_interview_data(
                conversation_state['user_data'],
                conversation_state['question_answers']
            )
            return f"✅ Thank you! All answers are been saved. Our recruitment Team will contact you shortly", False, False
        else:
            # Ask next question
            next_q = conversation_state['generated_questions'][len(conversation_state['question_answers'])]
            return f"{next_q[0]}:\n{next_q[1]}", True, True

    # Normal information collection
    current_field = information_steps[conversation_state['current_step']][1]
    conversation_state['user_data'][current_field] = message

    # Move to next question or show summary
    if conversation_state['current_step'] < len(information_steps) - 1:
        conversation_state['current_step'] += 1
        return information_steps[conversation_state['current_step']][0], False, False
    else:
        # Generate questions
        question_text = generate_questions(
            conversation_state['user_data']['tech_stack'],
            conversation_state['user_data']['experience']
        )
        conversation_state['generated_questions'] = parse_questions(question_text)

        # Start asking questions
        conversation_state['awaiting_answers'] = True
        first_q = conversation_state['generated_questions'][0]
        return f"🔍 Technical Questions:\n\n{first_q[0]}:\n1. {first_q[1]}", True, True

# Create the chat interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""# 🧑‍💼 TalentScout Interview Assistant""")

    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=500)
            msg = gr.Textbox(placeholder="Type your answer here...")

        with gr.Column(scale=1):
            gr.Markdown("### Progress")
            progress_bar = gr.Slider(
                maximum=len(information_steps) + 20,  # Extra for questions
                interactive=False
            )
            with gr.Row():
                submit_btn = gr.Button("📩 Submit Answer", visible=False)
                clear_btn = gr.Button("🔄 Start Over")

    # Chat functions
    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        response, show_submit, is_question = respond(history[-1][0], history[:-1])

        # Calculate progress
        if conversation_state['awaiting_answers']:
            progress = len(information_steps) + len(conversation_state['question_answers'])
        else:
            progress = conversation_state['current_step']

        return history + [[None, response]], gr.update(visible=show_submit), progress

    def submit_answer(history):
        # Treat submit same as pressing enter
        return "", history

    # Event handlers
    msg.submit(
        user,
        [msg, chatbot],
        [msg, chatbot],
        queue=False
    ).then(
        bot,
        chatbot,
        [chatbot, submit_btn, progress_bar]
    )

    submit_btn.click(
        submit_answer,
        None,
        [msg, chatbot],
        queue=False
    ).then(
        bot,
        chatbot,
        [chatbot, submit_btn, progress_bar]
    )

    clear_btn.click(
        lambda: (None, False, 0),
        None,
        [chatbot, submit_btn, progress_bar],
        queue=False
    )

demo.launch()

<ipython-input-3-07f2117f2601>:163: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 1 arguments for function <function submit_answer at 0x7ecb05eb6f20>, received 0.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected at least 1 arguments for function <function submit_answer at 0x7ecb05eb6f20>, received 0.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://badcb599b4991f6ef0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
